This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [2]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [30]:
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()

def mprint(val: str):
    display(Markdown(val))

---

# Tools and more

Before showning an example using a `Chain`, it will be helpful to understand the various tools that can be used in a chain.

## DuckDuckGo Search

We can use the DuckDuckGo search engine to retrieve the top URLs associated with a search query. The `DuckDuckGoSearch` object is a callable object that returns a list of dictionaries. Each item in the list corresponds to a search result.

In [1]:
from llm_chain.tools import DuckDuckGoSearch

duckduckgo_search = DuckDuckGoSearch(top_n=3)
duckduckgo_search("What is a large language model?")

[{'title': 'What is a large language model and how does it work? - Fast Company',
  'href': 'https://www.fastcompany.com/90884581/what-is-a-large-language-model',
  'body': 'Large language models are the foundational technology behind recent artificial intelligence advancements like ChatGPT.'},
 {'title': 'What are LLMs, and how are they used in generative AI?',
  'href': 'https://www.computerworld.com/article/3697649/what-are-large-language-models-and-how-are-they-used-in-generative-ai.html',
  'body': "Large language models are the algorithmic basis for chatbots like OpenAI's ChatGPT and Google's Bard. The technology is tied back to billions — even trillions — of parameters that can make them..."},
 {'title': 'What are Large Language Models - MachineLearningMastery.com',
  'href': 'https://machinelearningmastery.com/what-are-large-language-models/',
  'body': 'Large language models (LLMs) are recent advances in deep learning models to work on human languages. Some great use case of L

We also have access to the search history via the `history` property.

In [3]:
print(duckduckgo_search.history[0].query)
print(duckduckgo_search.history[0].results)

What is a large language model?
[{'title': 'What is a large language model and how does it work? - Fast Company', 'href': 'https://www.fastcompany.com/90884581/what-is-a-large-language-model', 'body': 'Large language models are the foundational technology behind recent artificial intelligence advancements like ChatGPT.'}, {'title': 'What are LLMs, and how are they used in generative AI?', 'href': 'https://www.computerworld.com/article/3697649/what-are-large-language-models-and-how-are-they-used-in-generative-ai.html', 'body': "Large language models are the algorithmic basis for chatbots like OpenAI's ChatGPT and Google's Bard. The technology is tied back to billions — even trillions — of parameters that can make them..."}, {'title': 'What are Large Language Models - MachineLearningMastery.com', 'href': 'https://machinelearningmastery.com/what-are-large-language-models/', 'body': 'Large language models (LLMs) are recent advances in deep learning models to work on human languages. Some g

---

# Chains

**A `Chain` consists of individual links. Each link can be thought of as a task in a workflow (e.g. document search, web search, or chat model). Each link is a callable (either a function or a callable object) where the output of one link is the input of the next link. So a `Chain` is a simple mechanism that takes an input and sends the input to the first link, and the propegates the output of the first link to the second link, until the end of the chain is reached, and returns the final result.** 

A `Chain` object aggregates the history (messages, usage, etc) of all of the links. More specifically, it aggregiates all of the lists of `Record` objects for any link that has a `history` property, such as the OpenAIChat class. The `Chain` class provides various history properties (e.g. `history`, `usage_history`, `message_history`) depending on the type of `Record` returned by each link's history property (e.g. the `usage_history` property returns all of the `UsageRecord` objects across any link whose `history` property returns `UsageRecord`s).

## Chat model w/ web search 

In the following example, we will ask a chat model a question, and provide the model with additional context based on the most relevant text we find in a web-search. For the sake of the example, we will ask the chat model to the answer it provides and summarize that answer.  The workflow is defined as follows:


|   input   |  output  |   link/task   |
|-----------|----------|----------|
|  `None` |   str    |  Ask a question: e.g. `"What is the meaning of life?"`  |
|    str    |   urls   |  Do a web-search via `DuckDuckGo` based on the question.  |
|   urls    | Documents|  Take the urls, scrape the corresponding web-pages, and convert to a list of `Document` objects  |
| Documents | Documents|  Split the `Document` objects into smaller chunks (also `Document` objects)  |
| Documents | `None` |  Store the Documents in a `Chroma` document index so that we can lookup the most relevant documents |
| `None`  |   str    |  Return the original question `What is the meaning of life?` |
|    str    |   str    |  Take the original question, lookup the most relevant documents, and construct the corresponding prompt with the documents injected into the prompt. |
|    str    |   str    |  Pass the prompt to the chat model; receive a response |
|    str    |   str    |  Construct a new prompt, containing the response and a request to summarize the response. |
|    str    |   str    |  Pass the new prompt to the chat model; receive a response |

**Notice how the output of one link matches the input of the next link.**



### Initial objects in chain

Let's define the objects and functions we are going to include in the chain. We're including many links/tasks so that we can see how flexible our chain can be.

#### Quick note on the `Value` object.

The only bit of magic below is that we're using a `Value` object to cache the initial question, feed it into the web-search, and then inject it back into the chain at a later point (passing it to the prompt-template). A `Value` object is a callable that, when called with a value caches and returns the same value, and called without a value, simply returns the previously cached value. When you understand what it's doing, it's not magic at all. Here's a simple example:

In [11]:
from llm_chain.base import Value
cache = Value()
result = cache("This is a value")  # calling the object caches and returns the value
print(result)  # the return value is whatever is passed in
print(cache())  # calling the object returns the cached value
result = cache("new value")
print(result)
print(cache())

This is a value
This is a value
new value
new value


In [4]:
from llm_chain.base import Document, Chain, Value
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import DuckDuckGoSearch, scrape_url, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.prompt_templates import DocSearchTemplate

# Seach DuckDuckGo based on initial question (passed into the chain)
duckduckgo_search = DuckDuckGoSearch(top_n=3)

# define a function that takes the links from the web-search, scrapes the web-pages,
# and then creates Document objects from the text of each web-page
def scrape_urls(search_results):
    """
    For each url (i.e. `href` in `search_results`):
    - extracts text
    - replace new-lines with spaces
    - create a Document object
    """
    return [
        Document(content=scrape_url(x['href']).replace('\n', ' '))
        for x in search_results
    ]

# Embeddings model used for document index/search (the documents created from the web-search)
embeddings_model = OpenAIEmbeddings(model_name='text-embedding-ada-002')
document_index = ChromaDocumentIndex(embeddings_model=embeddings_model, n_results=3)
# DocSearchTemplate uses the ChromaDocumentIndex object to search for the most relevant documents
# (from the web-search) based on the intitial question (which it takes as an input)
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=3)
# Create a chat model without streaming.
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')
# Create a chat model with streaming enabled via callback.
streaming_chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|'),
)

# A `Value` object is a simple caching mechanism. It is a callable object that, 
# when called with a value passed in, caches the value in an instance field and returns the value;
# and when called without a value passed in returns the cached value stored in the instance field.
# In this case, it's a way for us to pass our question to the `DuckDuckGoSearch` object and then
# also pass it to the `DocSearchTemplate` object in the middle of the chain.
initial_question = Value()
# This simple function takes the response from the original chat model and creates a prompt that
# asks the model to summarize the response.
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

### Running the Chain

In [5]:
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    initial_question,
    duckduckgo_search,
    scrape_urls,
    split_documents,  # split web-pages into smaller chunks; defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    initial_question,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])
# the value passed into `chain()` is passed to the `initial_question` object (which
# is `Value` object and so it caches the value passed in and also returns it) which then gets
# passed to the `duckduckgo_search` object, and so on.
# the response of the final model is streamed, but should also match the response returned
response = chain("What is the meaning of life?")

The| meaning| of| life| is| subjective| and| each| person| must| answer| it| for| themselves|;| there| is| no| one| answer|.|

In [6]:
response

'The meaning of life is subjective and each person must answer it for themselves; there is no one answer.'

---

### Total costs and usage

The `Chain` object aggregates the costs/usage across all links that have a `history` property where that `history` property returns `UsageRecord` objects. In other words, any link that tracks its own history automatically gets counted towards the total usage within the Chain.

In [45]:
print(f"Cost:   ${chain.cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost:   $0.0054
Tokens: 45,674


### History

Similar to tracking the costs/usage, we can dig into the history at a more granular level.

As you can see below, the `history` property returns a list containing:

- a `SearchRecord` object capturing our original search query and results
- two `EmbeddingsRecord` records; the first corresponds to getting the embeddings of all of the chunks from the web-pages; the second corresponds to getting the embedding of our original question so that we can find the most relevant chunks; the `EmbeddingsRecord` is a `UsageRecord` so it contains costs/usage
- two `MessageRecord` records; the first corresponds to the first question (prompt & response) we made to the chat model; the second corresponds to our second query to the chat model asking it to summarize the first response; the `MessageRecord` is a `UsageRecord` so it contains costs/usage

In [31]:
chain.history

[SearchRecord(uuid='5a7f9a96-4d13-428f-8145-9aad6be50f88', timestamp='2023-06-24 02:29:01.768', metadata={}, query='What is the meaning of life?', results=[{'title': 'Meaning of life - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Meaning_of_life', 'body': 'The meaning of life, or the answer to the question: "What is the meaning of life?", pertains to the significance of living or existence in general. Many other related questions include: "Why are we here?", "What is life all about?", or "What is the purpose of existence?"'}, {'title': 'Life Definition & Meaning - Merriam-Webster', 'href': 'https://www.merriam-webster.com/dictionary/life', 'body': 'The meaning of LIFE is the quality that distinguishes a vital and functional being from a dead body. How to use life in a sentence.'}, {'title': 'What Is the Meaning of Life According to Positive Psychology', 'href': 'https://positivepsychology.com/meaning-of-life-positive-psychology/', 'body': '"The sole meaning of life is to serve hu

In [32]:
print(chain.history[3])
print(chain.history[4])

timestamp: 2023-06-24 02:29:07.919; prompt: "Answer the question ..."; response: "The meaning of life ...";  cost: $0.000724; total_tokens: 467; metadata: {'model_name': 'gpt-3.5-turbo'}
timestamp: 2023-06-24 02:29:09.724; prompt: "Summarize the follow..."; response: "The meaning of life ...";  cost: $0.000160; total_tokens: 100; metadata: {'model_name': 'gpt-3.5-turbo'}


#### `message_history`

We can use the `message_history` property which simply returns all of the history items that are of type `MessageRecord`

In [39]:
chain.message_history

[MessageRecord(uuid='e4c3f2e6-5a65-41e6-9f34-7e7b7f5d52ad', timestamp='2023-06-24 02:29:07.919', metadata={'model_name': 'gpt-3.5-turbo'}, total_tokens=467, cost=0.000724, prompt="\nAnswer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.\n\nHere is the information:\n\n```\nthe meaning of life at all? To what purpose is it played, this farce in which everything that is essential is irrevocably fixed and determined?[5]Questions about the meaning of life have been expressed in a broad variety of other ways, including: What is the meaning of life? What's it all about? Who are we?[6][7][8] Why are we here? What are we here for?[9][10][11] What is the origin of life?[12] What is the nature of life? What is the nature of reality?[12][13][14] What is the purpose of life? \n\nto the meaning of life is too profound to be known and understood.[189] You will never live if you are looking for the meaning of life.[161] The mea

In [40]:
print(chain.message_history[0])  # same as chain.history[3]
print(chain.message_history[1])  # same as chain.history[4]

timestamp: 2023-06-24 02:29:07.919; prompt: "Answer the question ..."; response: "The meaning of life ...";  cost: $0.000724; total_tokens: 467; metadata: {'model_name': 'gpt-3.5-turbo'}
timestamp: 2023-06-24 02:29:09.724; prompt: "Summarize the follow..."; response: "The meaning of life ...";  cost: $0.000160; total_tokens: 100; metadata: {'model_name': 'gpt-3.5-turbo'}


#### First prompt to the chat model

You can see below that we used three chunks from our original web-search in the first prompt we sent to ChatGPT.

In [41]:
mprint(chain.message_history[0].prompt)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
the meaning of life at all? To what purpose is it played, this farce in which everything that is essential is irrevocably fixed and determined?[5]Questions about the meaning of life have been expressed in a broad variety of other ways, including: What is the meaning of life? What's it all about? Who are we?[6][7][8] Why are we here? What are we here for?[9][10][11] What is the origin of life?[12] What is the nature of life? What is the nature of reality?[12][13][14] What is the purpose of life? 

to the meaning of life is too profound to be known and understood.[189] You will never live if you are looking for the meaning of life.[161] The meaning of life is to forget about the search for the meaning of life.[161] Ultimately, a person should not ask what the meaning of their life is, but rather must recognize that it is they themselves who are asked. In a word, each person is questioned by life; and they can only answer to life by answering for their own life; to life they can only respon

d the most is “What is the meaning of life?” “Ultimately, man should not ask what the meaning of his life is, but rather must recognize that it is he who is asked. In a word, each man is questioned by life; and he can only answer to life by answering for his own life; to life he can only respond by being responsible.” Viktor Frankl We are all hungry for meaning, for purpose, for the feeling that our life is worth more than the sum of its parts. Luckily, humans are resourceful – we have infinite 
```

Here is the question:

What is the meaning of life?


#### First response from the chat model

In [42]:
mprint("> " + chain.message_history[0].response)

> The meaning of life is a profound question that has been expressed in various ways, but ultimately, it is up to each individual to answer for their own life and be responsible for it. There is no one definitive answer to this question.

#### Second prompt to the chat model

In [43]:
mprint(chain.message_history[1].prompt)

Summarize the following in less than 20 words: "The meaning of life is a profound question that has been expressed in various ways, but ultimately, it is up to each individual to answer for their own life and be responsible for it. There is no one definitive answer to this question."

#### Second response from the chat model

In [44]:
mprint("> " + chain.message_history[1].response)

> The meaning of life is subjective and each person must answer it for themselves; there is no one answer.

---